<a href="https://colab.research.google.com/github/Rami-RK/pytorch_tf_app_on_hf_spaces/blob/main/Gradio_app_with_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Saving a Tensorflow trained model and integration with Gradio APP on HF Spaces**

## Learning Objectives:

At the end of the experiment, you will be able to:

1. Understand Hugging Face Spaces
2. Deploy custom model on spaces with Gradio

## Building an Image Classification model


We know how to build a simple CNN, let's build and train one to solve an image classification problem.

We will work with the cats-vs-dogs dataset to classify whether a given image is that of a cat or a dog .i.e a  binary classification problem.

### Import libraries

In [1]:
import os
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from matplotlib.image import imread
import numpy as np
from tensorflow.keras.utils import image_dataset_from_directory

We have already uploaded the dataset into structured folders. You simply need to download it from our repository.

In [2]:
#@title Download the data
!wget -O cats_vs_dogs_small.zip -qq https://www.dropbox.com/scl/fi/wiaspjsrue17jbtkp6vga/cats_vs_dogs_small.zip?rlkey=4798vwv7v75bihwjxye0tr7c3&dl=0
!unzip -qq '/content/cats_vs_dogs_small.zip'

In [3]:
# defining path names for futur use
data_dir = '/content/cats_vs_dogs_small'

train_path = data_dir + '/train'
validation_path = data_dir + '/validation'
test_path = data_dir + '/test'

### Converting the image dataset into a workable format

We have the images in folders. We need to make it into a workable dataset:
  * Which has labels
  * All the images have the same size

For this, we will use the utility [**image_dataset_from_directory**](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory).

Calling image_dataset_from_directory(main_directory, labels='inferred') will return a tf.data.Dataset that yields batches of images from the subdirectories class_a and class_b, together with labels 0 and 1 (0 corresponding to class_a and 1 corresponding to class_b).

In [4]:
train_dataset = image_dataset_from_directory(
               train_path,
                image_size=(180, 180), # Resize the images to (180,180)
                batch_size=32,
                class_names=['cat','dog'],)
validation_dataset = image_dataset_from_directory(
                      validation_path,
                      image_size=(180, 180),
                      batch_size=32,
                class_names=['cat','dog'],)
test_dataset = image_dataset_from_directory(
                test_path,
                image_size=(180, 180),
                batch_size=32,
                class_names=['cat','dog'],)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [5]:
print(f"train_dataset = {train_dataset}")

train_dataset = <_BatchDataset element_spec=(TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [ ]:
# Verify batch size
for data_batch, labels_batch in train_dataset:
  print("data batch shape:", data_batch.shape)
  print("labels batch shape:", labels_batch.shape)
  break
# Q: What is the batch size of each mini-batch? A: 32

data batch shape: (32, 180, 180, 3)
labels batch shape: (32,)


#### Call Back Function

In [ ]:
# Define a function to return a commmonly used callback_list
def def_callbacks(filepath, mod_chk_mon = "val_loss", tensorboard = True, earlystop = 0 ):
    callback_list = []

    # Defualt callback
    callback_list.append(keras.callbacks.ModelCheckpoint(filepath,
                                         save_best_only = True,
                                         monitor=mod_chk_mon))
    if tensorboard:
      log_dir = "tensorLog_" + filepath
      callback_list.append(keras.callbacks.TensorBoard(log_dir=log_dir))

    if earlystop>0:
       callback_list.append(keras.callbacks.EarlyStopping(patience=earlystop))

    return callback_list

### Data Augmentation

The small dataset can cause a high variance estimation of model performance

Q: How to overcome this and get a more robust model?

Now, we want to avoid this problem altogether by artificially (and cleverly) producing new data from the already available data.

For this, we perform **data augmentation**.

Data augmentation is another regularization method. What other methods did we see in the last tutorial?

Data augmentation takes the approach of generating more training data from existing training samples by augmenting the samples via a number of random transformations that yield a believable-looking image. Common transformations include:
  * Flipping the image
  * Rotating the image
  * Zooming in/out of the image

See some sample images below after augmentation:

![picture](https://drive.google.com/uc?export=view&id=1HRhsHEHtcVptNVMF1EbCGiZX5XuTdrs5)

In [ ]:
# Performing the data augmentation as series of transformations
def get_data_augmented(flip="horizontal",rotation=0.1,zoom=0.2):
    data_augmentation = keras.Sequential([
      keras.layers.RandomFlip(flip),
      keras.layers.RandomRotation(rotation),
      keras.layers.RandomZoom(zoom)])
    return data_augmentation
# Q: what does the above function return? A: A sequence of layers

data_augmentation = get_data_augmented()


In [ ]:
inputs = keras.Input(shape=(180, 180, 3))
# Augmenting data - Transformations of images by random factors
# so the the network never sees the same data twice
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)     # Q: Dropout is a _______ method
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
PARTIAL_RUN = False
epochs = 144
if PARTIAL_RUN:
  epochs = 2
history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=validation_dataset,
    callbacks=def_callbacks("convnet_from_scratch_with_augmentation.keras"))

Epoch 1/144
63/63 [==============================] - 12s 90ms/step - loss: 0.6987 - accuracy: 0.4820 - val_loss: 0.6927 - val_accuracy: 0.5000
Epoch 2/144
63/63 [==============================] - 7s 100ms/step - loss: 0.7304 - accuracy: 0.4950 - val_loss: 0.6960 - val_accuracy: 0.5000
Epoch 3/144
63/63 [==============================] - 5s 72ms/step - loss: 0.6930 - accuracy: 0.5060 - val_loss: 0.6910 - val_accuracy: 0.5040
Epoch 4/144
63/63 [==============================] - 6s 89ms/step - loss: 0.6936 - accuracy: 0.5460 - val_loss: 0.6787 - val_accuracy: 0.6480
Epoch 5/144
63/63 [==============================] - 5s 69ms/step - loss: 0.6895 - accuracy: 0.5940 - val_loss: 0.6437 - val_accuracy: 0.6080
Epoch 6/144
63/63 [==============================] - 5s 70ms/step - loss: 0.6578 - accuracy: 0.6215 - val_loss: 0.6277 - val_accuracy: 0.6320
Epoch 7/144
63/63 [==============================] - 6s 86ms/step - loss: 0.6405 - accuracy: 0.6415 - val_loss: 0.6573 - val_accuracy: 0.5820
Epoc

In [ ]:
test_model = keras.models.load_model(
            "convnet_from_scratch_with_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

63/63 [==============================] - 4s 49ms/step - loss: 0.4372 - accuracy: 0.8270
Test accuracy: 0.827


With data augmentation, we roughly get **82-85%** accuracy.

### Checking the size of the model

In [ ]:
from pathlib import Path

# Get the model size in bytes then convert to megabytes
pretrained_model_size = Path("convnet_from_scratch_with_augmentation.keras").stat().st_size // (1024*1024) # division converts bytes to megabytes (roughly)
print(f"Pretrained model size: {pretrained_model_size} MB")

Pretrained model size: 7 MB


### Defining the prediction function

In [ ]:
import random
from PIL import Image
from timeit import default_timer as timer

In [ ]:
MODEL = test_model

In [ ]:
def predict(img):

    # Start the timer
    start_time = timer()

    # Reading the image and size transformation
    features = Image.open(img)
    features = features.resize((180, 180))
    features = np.array(features).reshape(1, 180,180,3)

    # Create a prediction label and prediction probability dictionary for each prediction class
    # This is the required format for Gradio's output parameter
    pred_labels_and_probs = {'dog' if MODEL.predict(features)> 0.5 else 'cat':float(MODEL.predict(features))}

    # Calculate the prediction time
    pred_time = round(timer() - start_time, 5)

    # Return the prediction dictionary and prediction time
    return pred_labels_and_probs, pred_time

In [ ]:
predict('/content/cats_vs_dogs_small/test/cat/cat.1502.jpg')

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

# Create title, description and article strings
title = "Classification Demo"
description = "Cat/Dog classification Tensorflow model with Augmentted small dataset"

# Create the Gradio demo
demo = gr.Interface(fn=predict, # mapping function from input to output
                    inputs=gr.Image(type='filepath'), # what are the inputs?
                    outputs=[gr.Label(label="Predictions"), # what are the outputs?
                             gr.Number(label="Prediction time (s)")], # our fn has two outputs, therefore we have two outputs
                    #examples=example_list,
                    title=title,
                    description=description,)

# Launch the demo!
demo.launch(debug=False, # print errors locally?
            share=True) # generate a publically shareable URL?

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e73fd6212f894e5fba.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


### File structure for uploading on HuggingFace
To upload our demo Gradio app, we'll want to put everything relating to it into a single directory.

For example, our demo might live at the path `demo/binary_Classification/` with the file structure:

```
demo/
# └── binary_Classification/
    ├── convnet_from_scratch_with_augmentation.keras
    ├── app.py
    ├── examples/
    │   ├── example_1.jpg
    │   ├── example_2.jpg
    │   └── example_3.jpg
    ├── ____.py
    └── requirements.txt
```

Where:
* `convnet_from_scratch_with_augmentation.keras` is our trained Tensorflow model file.
* `app.py` contains our Gradio app (similar to the code that launched the app).
    * **Note:** `app.py` is the default filename used for Hugging Face Spaces, if you deploy your app there, Spaces will by default look for a file called `app.py` to run. This is changable in settings.
* `examples/` contains example images to use with our Gradio app.
* `model.py` contains the model defintion as well as any transforms assosciated with the model.
* `requirements.txt` contains the dependencies to run our app such as `tensorflow`, `numpy` and `gradio`.



### Steps for Uploading Classification app on Huggingface spaces

 **Note:** The following series of steps uses a Git (a file tracking system) workflow.

1. [Sign up](https://huggingface.co/join) for a Hugging Face account.
2. Start a new Hugging Face Space by going to your profile and then [clicking "New Space"](https://huggingface.co/new-space).
    * **Note:** A Space in Hugging Face is also known as a "code repository" (a place to store your code/files) or "repo" for short.
3. Give the Space a name, for example, mine is called `Ramendra/image_classification`, you can see it here: https://huggingface.co/spaces/Ramendra/image_classification
4. Select a license (I used [MIT](https://opensource.org/licenses/MIT)).
5. Select Gradio as the Space SDK (software development kit).
   * **Note:** You can use other options such as Streamlit but since our app is built with Gradio, we'll stick with that.
6. Choose whether your Space is it's public or private (I selected public since I'd like my Space to be available to others).
7. Click "Create Space".
8. Clone the repo locally by running something like: `git clone https://huggingface.co/spaces/[YOUR_USERNAME]/[YOUR_SPACE_NAME]` in terminal or command prompt.
    * **Note:** You can also add files via uploading them under the "Files and versions" tab.
9. Copy/move the contents of the application files/folder to the cloned repo folder.

  `git remote -v`

10. To upload and track larger files (e.g. files over 10MB or in our case, our PyTorch model file) you'll need to [install Git LFS](https://git-lfs.github.com/) (which stands for "git large file storage").

  `git lfs install`

  Track the files over 10MB with Git LFS with `git lfs track "*.file_extension"`.

  `git lfs track "convnet_from_scratch_with_augmentation.keras"`


13. Track `.gitattributes` (automatically created when cloning from HuggingFace, this file will help ensure our larger files are tracked with Git LFS). You can see an example `.gitattributes` file on the spaces repo unders files.
    * `git add .gitattributes`

14. Add the rest of the `foodvision_mini` app files and commit them with:
    * `git add *`
    * `git commit -m "first commit"`
15. Push (upload) the files to Hugging Face:
    * `git push`
16. Wait 3-5 minutes for the build to happen (future builds are faster) and your app to become live!

If everything worked, you should see a live running example of our classification demo like the one here: https://huggingface.co/spaces/Ramendra/image_classification

### Notebook for creating app.py file

This [Notebook](https://colab.research.google.com/drive/13X2E9v7GxryXyT39R5CzxrNwxfA6KMFJ?usp=sharing) contains only trained model uploading and inference scripts along with Gradio implementation. **No training scripts.**